<a href="https://colab.research.google.com/github/SaniyaBubere/Lead-Scoring-Model/blob/main/LeadScoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The provided dataset includes information about leads generated for rental properties. Each row represents a lead and provides various details such as agent ID, lead status, lost reason, budget, lease duration, move-in date, source of lead, destination city and country, desired room type, and lead ID.

There are four rows in the dataset, and each row represents a lost lead. The lost reasons for the leads are "Not responding" and "Low budget". One lead has a specified budget of "£121 - £180 per week" and a lease duration of "Full Year Course Stay 40 - 44 weeks". The move-in date for one lead is "31/08/22".

The dataset also includes information about the source of the leads such as the source website, source city, source country, UTM source, and UTM medium. Finally, each lead has a unique lead ID.

In the context of sales and marketing, a lead is a potential customer or prospect who has shown interest in a company's products or services.

Lead status refers to the current stage of a lead in the sales process. It describes whether a lead is a potential customer, a lost opportunity, or a converted customer.


Lease refers to a contractual agreement between two parties, whereby the owner of a property (lessor) allows another party (lessee) to use the property for a specified period of time in exchange for rent

In [50]:
import pandas as pd
import numpy as np

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
df=pd.read_excel("/content/drive/MyDrive/Data_Science_Internship.xlsx")


In [53]:
df.head()

,Unnamed: 0,Agent_id,status,lost_reason,budget,lease,movein,source,source_city,source_country,utm_source,utm_medium,des_city,des_country,room_type,lead_id
0,0.0,1deba9e96f404694373de9749ddd1ca8aa7bb823145a6f...,LOST,Not responding,NaN,NaN,NaT,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,ecc0e7dc084f141b29479058967d0bc07dee25d9690a98...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,cd5dc0d9393f3980d11d4ba6f88f8110c2b7a7f7796307...
1,1.0,299ae77a4ef350ae0dd37d6bba1c002d03444fb1edb236...,LOST,Low budget,NaN,NaN,NaT,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,5372372f3bf5896820cb2819300c3e681820d82c6efc54...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,b94693673a5f7178d1b114e4004ad52377d3244dd24a3d...
2,2.0,c213697430c006013012dd2aca82dd9732aa0a1a6bca13...,LOST,Not responding,£121 - £180 Per Week,Full Year Course Stay 40 - 44 weeks,2022-08-31,7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2...,9b8cc3c63cdf447e463c11544924bf027945cbd29675f7...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,11ab03a1a8c367191355c152f39fe28cae5e426fce49ef...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,Ensuite,96ea4e2bf04496c044745938c0299c264c3f4ba079e572...
3,3.0,eac9815a500f908736d303e23aa227f0957177b0e6756b...,LOST,Low budget,0-0,0.0,NaT,ba2d0a29556ac20f86f45e4543c0825428cba33fd7a9ea...,a5f0d2d08eb0592087e3a3a2f9c1ba2c67cc30f2efd2bd...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,19372fa44c57a01c37a5a8418779ca3d99b0b59731fb35...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,1d2b34d8add02a182a4129023766ca4585a8ddced0e5b3...
4,4.0,1deba9e96f404694373de9749ddd1ca8aa7bb823145a6f...,LOST,Junk lead,NaN,NaN,NaT,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,NaN,fc10fffd29cfbe93c55158fb47752a7501c211d253468c...


In [54]:
df["status"].unique()

array(['LOST', 'WON', 'OPPORTUNITY', 'CONTACTED', 'PROCESSING',
       'IMPORTANT'], dtype=object)

In [55]:
df = df[df["status"].isin(["WON","LOST"])]

In [56]:
value='9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0' 

In [57]:
ispresent=df.isin([value]).any().any()

In [58]:
if ispresent:
  print("the value",value)
else:
  print("not")

the value 9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0


In [59]:
count=df.isin([value]).sum()
count

Unnamed: 0           0
Agent_id             0
status               0
lost_reason          0
budget               0
lease                0
movein               0
source            5951
source_city       8831
source_country    8622
utm_source          61
utm_medium        3184
des_city          2529
des_country       2529
room_type            0
lead_id              0
dtype: int64

In [60]:
# Replace missing values with a unique value
df.replace('9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0', np.nan, inplace=True)

In [61]:
count=df.isin([value]).sum()
count

Unnamed: 0        0
Agent_id          0
status            0
lost_reason       0
budget            0
lease             0
movein            0
source            0
source_city       0
source_country    0
utm_source        0
utm_medium        0
des_city          0
des_country       0
room_type         0
lead_id           0
dtype: int64

In [62]:
# Convert all columns to categorical type
df = df.astype('category')

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46317 entries, 0 to 46607
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Unnamed: 0      46317 non-null  category
 1   Agent_id        46317 non-null  category
 2   status          46317 non-null  category
 3   lost_reason     43244 non-null  category
 4   budget          42623 non-null  category
 5   lease           43981 non-null  category
 6   movein          32707 non-null  category
 7   source          40366 non-null  category
 8   source_city     37486 non-null  category
 9   source_country  37695 non-null  category
 10  utm_source      46256 non-null  category
 11  utm_medium      43133 non-null  category
 12  des_city        43788 non-null  category
 13  des_country     43788 non-null  category
 14  room_type       22826 non-null  category
 15  lead_id         46317 non-null  category
dtypes: category(16)
memory usage: 4.5 MB


In [64]:
# Handle missing values by filling them with mode
df = df.fillna(df.mode().iloc[0])

In [65]:
df.isna().sum()

Unnamed: 0        0
Agent_id          0
status            0
lost_reason       0
budget            0
lease             0
movein            0
source            0
source_city       0
source_country    0
utm_source        0
utm_medium        0
des_city          0
des_country       0
room_type         0
lead_id           0
dtype: int64

In [66]:
df.empty

False

# Feature Selection:


In [67]:
import seaborn as sns
import matplotlib.pyplot as plt

In [74]:
df.to_numpy()

array([[0.0,
        '1deba9e96f404694373de9749ddd1ca8aa7bb823145a6fe3491dec605a526a92',
        'LOST', ...,
        '8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512ff098eaa56a4d5fb19',
        'Ensuite',
        'cd5dc0d9393f3980d11d4ba6f88f8110c2b7a7f7796307af5f50126395bee0dd'],
       [1.0,
        '299ae77a4ef350ae0dd37d6bba1c002d03444fb1edb236b341962dec14cb4bb4',
        'LOST', ...,
        '8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512ff098eaa56a4d5fb19',
        'Ensuite',
        'b94693673a5f7178d1b114e4004ad52377d3244dd24a3d7a4ff6f56f21cae7e3'],
       [2.0,
        'c213697430c006013012dd2aca82dd9732aa0a1a6bca1371c92f18fd11d9bc5f',
        'LOST', ...,
        '8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512ff098eaa56a4d5fb19',
        'Ensuite',
        '96ea4e2bf04496c044745938c0299c264c3f4ba079e57267aab3e807e1008861'],
       ...,
       [46605.0,
        '1134c0a7d44fdae1afd7f1f64e2789496784095ca0a050947d659c256d331d34',
        'LOST', ...,
        '8d23a6e37e0a6431a8f

ValueError: ignored

In [77]:
print(df.corr())
  
# plotting correlation heatmap
dataplot = sns.heatmap(df.corr(), cmap="YlGnBu")
  
# displaying heatmap
plt.show()

Empty DataFrame
Columns: []
Index: []


ValueError: ignored

In [78]:
# Remove rows with missing values
df = df.dropna()

# Compute the correlation matrix
corr = df.corr()

# Plot the heatmap
dataplot = sns.heatmap(corr, cmap="YlGnBu", annot=True)

# Display the heatmap
plt.show()


ValueError: ignored

In [76]:
df.shape

(46317, 16)

In [80]:
df=pd.to_numeric()

TypeError: ignored

In [70]:
corr_matrix = df.corr()
corr_matrix.style.background_gradient()
plt.figure(figsize=(10,10))
cor = df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)

ValueError: ignored

<Figure size 720x720 with 0 Axes>